In [20]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

TRAINING_DATA_PATH="/Users/slouvan/sandbox/cross-domain/data/atis-2.train.conll"
DEV_DATA_PATH="/Users/slouvan/sandbox/cross-domain/data/atis-2.dev.conll"
TEST_DATA_PATH="/Users/slouvan/sandbox/cross-domain/data/atis.test.iob.conll"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import anago.utils
import anago
from anago.reader import load_data_and_labels

In [22]:
model = anago.Sequence()
model = model.load_best_model("/Users/slouvan/sandbox/cross-domain/models/anago/CONLL_2003_BASE_MODEL_RANDOM")
x_train , y_train = load_data_and_labels(TRAINING_DATA_PATH)
x_valid   , y_valid   = load_data_and_labels(DEV_DATA_PATH)
x_test  , y_test  = load_data_and_labels(TEST_DATA_PATH)



PREPROCESSOR FILE : /Users/slouvan/sandbox/cross-domain/models/anago/CONLL_2003_BASE_MODEL_RANDOM/preprocessor.pkl
THE VOCABULARY SIZE 1 IS 17495
Hello
Loading the best model from /Users/slouvan/sandbox/cross-domain/models/anago/CONLL_2003_BASE_MODEL_RANDOM/model_weights_07_88.72.h5 


In [23]:
model.update_training_config("/Users/slouvan/sandbox/anago/config/atis_transfer_learning.config")
model.re_train(x_train, y_train, x_valid, y_valid)

{'optimizer': 'adam', 'max_epoch': 2, 'clip_gradients': 5.0, 'batch_size': 20, 'lr_decay': 0.9, 'patience': 3, 'max_checkpoints_to_keep': 5, 'learning_rate': 0.001, 'early_stopping': 1, 'train_embeddings': 1}
Chopping off the top one
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, None, None)   0                                            
__________________________________________________________________________________________________
embedding_16 (Embedding)        (None, None, None, 2 1275        input_16[0][0]                   
__________________________________________________________________________________________________
lambda_15 (Lambda)              (None, None, 25)     0           embedding_16[0][0]               
_________________________________________________________________________

In [5]:
best_model = model.load_best_model()

In [6]:
model.model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None)   0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, None, 2 1275        input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None, 25)     0           embedding_2[0][0]                
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 25), (None,  5100        lambda_1[0][0]                   
__________________________________________________________________________________________________
lstm_2 (LS

In [7]:
model.model.layers.pop()
model.model.layers.pop()

In [8]:
model.model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None)   0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, None, 2 1275        input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None, 25)     0           embedding_2[0][0]                
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 25), (None,  5100        lambda_1[0][0]                   
__________________________________________________________________________________________________
lstm_2 (LS

In [ ]:
x = Dense(config.num_word_lstm_units, activation='relu')(model.model.layers[-1].output)
x = Dense(10)(x)
model.model.crf = ChainCRF()
pred = self.crf(x)

model2 = Model(input=in_img, output=[o])
model2.summary()

In [25]:


TRAIN_ATIS_PATH = "/Users/slouvan/sandbox/cross-domain/data/atis-2.train.conll"
x_train_atis , _ = load_data_and_labels("/Users/slouvan/sandbox/cross-domain/data/atis-2.train.conll")
x_train_conll, _, = load_data_and_labels("/Users/slouvan/sandbox/anago/data/conll2003/en/ner/train.lower.txt")
x_dev_atis, _, = load_data_and_labels("/Users/slouvan/sandbox/cross-domain/data/atis-2.dev.conll")


In [26]:
import itertools 

atis_word = set()
conll_word = set()
atis_dev_word = set()

for x in itertools.chain(*x_train_atis):
    atis_word.add(x)

for x in itertools.chain(*x_train_conll):
    conll_word.add(x)
    
for x in itertools.chain(*x_dev_atis):
    atis_dev_word.add(x)

In [28]:
len(atis_word), len(conll_word), len(atis_dev_word)

(869, 21010, 465)

In [34]:
in_atis_not_in_conll = atis_word - conll_word

In [35]:
len(in_atis_not_in_conll)

259

In [38]:
259.0/869

0.29804372842347526

In [31]:
in_atis_dev_not_in_atis_train = atis_dev_word - atis_word

In [32]:
len(in_atis_dev_not_in_atis_train)

31